## Lesson 0. Preparing

First, let's be sure that `bcc` is installed. See https://github.com/iovisor/bcc/blob/master/INSTALL.md if you need help.

We also need some helper function in order to run bcc examples in this notebook. Since most bcc examples will run until interrupted (Ctrl-C), we will create a helper function to run them for some time and then exit.

In [1]:
from __future__ import print_function
from contextlib import contextmanager
import signal

import bcc


class TimeoutError(Exception):
    pass


def alarm_handler(signum, frame):
    raise TimeoutError()
    

@contextmanager
def timeout(seconds=10):
    signal.signal(signal.SIGALRM, alarm_handler)
    signal.alarm(seconds)
    try:
        yield
    except TimeoutError:
        print('finished after {} seconds'.format(seconds))
    finally:
        # clear the alarm
        signal.alarm(0)

### Lesson 1. Hello World

Running this BPF program while executing some commands (ls, date, etc.) in another session/connection. It should print "Hello, World!" everytime a new process is created.

In [6]:
from bcc import BPF

prog = r'''
int kprobe__sys_clone(void *ctx) {
    bpf_trace_printk("Hello, World!\n");
    return 0;
}
'''

with BPF(text=prog) as b:
    with timeout():
        b.trace_print()

            bash-1541  [002] d... 24421.486005: : Hello, World!
            bash-1541  [002] d... 24423.366914: : Hello, World!
finished after 10 seconds


A bcc program is normally a mix of Python and C. The C code is the BPF program and will be compiled and attached to kernel probes to run. The Python code is bcc's helpers and other driver code to parse input arguments, print output, etc.

BPF code looks just like normal C source code, with mostly function definitions.
`kprobe__` is a special prefix that will attach the defined function to a kernel function, via kprobes. In this case, it will be attached to the `sys_clone()` function in kernel.

`bpf_trace_printk` is a [kernel facility](https://github.com/torvalds/linux/blob/510c8a899caf095cb13d09d203573deef15db2fe/kernel/trace/bpf_trace.c#L128) for printing to the common trace_pipe (`/sys/kernel/debug/tracing/trace_pipe`). It can be used for some quick examples but has limitations: 3 args max, only 1 `%s` and the pipe is globally shared, so concurrent programs will clash. A better I/O interface will be covered later.

> Side note: system calls are named sys_<something> in kernel source, e.g. sys_clone for clone() system call.

### Lesson 2. sys_sync()

In [ ]:
prog = r'''
int kprobe__sys_sync(void *ctx) {
    bpf_trace_printk("sys_sync() called\n");
    return 0;
}
'''

with BPF(text=prog) as b:
    print('tracing sys_sync()')
    with timeout():
        b.trace_print()

### Lesson 3. hello_fields

In [ ]:
prog = r'''
int hello(void *ctx) {
    bpf_trace_printk("Hello, World!\n");
    return 0;
}
'''

with BPF(text=prog) as b:
    b.attach_kprobe(event='sys_clone', fn_name='hello')
    
    # header
    print('%-18s %-16s %-6s %s' % ('TIME(s)', 'COMM', 'PID', 'MESSAGE'))
    
    with timeout():
        while True:
            try:
                task, pid, cpu, flags, ts, msg = b.trace_fields()
            except ValueError:
                pass
            else:
                print('%-18.9f %-16s %-6d %s' % (ts, task, pid, msg))

### Lesson 4. sync_timing

In [ ]:
prog = r'''
#include <uapi/linux/ptrace.h>
#include <linux/blkdev.h>

BPF_HASH(last);

int do_trace(struct pt_regs *ctx) {
    u64 ts, *tsp, delta, key = 0;
    
    // attempt to read stored timestamp
    tsp = last.lookup(&key);
    if (tsp != 0) {
        delta = bpf_ktime_get_ns() - *tsp;
        if (delta < 1000000000) {
            // output if time is less than 1 second
            bpf_trace_printk("%d\n", delta / 1000000);
        }
        last.delete(&key);
    }
    
    // update stored timestamp
    ts = bpf_ktime_get_ns();
    last.update(&key, &ts);
    return 0;
}
'''

with BPF(text=prog) as b:
    b.attach_kprobe(event='sys_sync', fn_name='do_trace')
    print('Tracing for quick sync\'...')
    
    start = 0
    with timeout(seconds=20):
        while True:
            task, pid, cpu, flags, ts, ms = b.trace_fields()
            if start == 0:
                start = ts
            ts = ts - start
            print('At time %.2f: s: multiple syncs detected, last %s ms ago' % (ts, ms))

### Lessson 5. sync_count.py

In [ ]:
prog = r'''
#include <uapi/linux/ptrace.h>
#include <linux/blkdev.h>

BPF_HASH(dict);

int do_trace(struct pt_regs *ctx) {
    u64 ts, *tsp, ts_key = 0;
    u64 count, *countp, count_key = 1;
    u64 delta;

    // counting calls
    countp = dict.lookup(&count_key);
    if (countp != 0) {
        count = *countp + 1;
        dict.delete(&count_key);
    } else {
        count = 1;
    }
    dict.update(&count_key, &count);
    
    // attempt to read stored timestamp
    tsp = dict.lookup(&ts_key);
    if (tsp != 0) {
        delta = bpf_ktime_get_ns() - *tsp;
        if (delta < 1000000000) {
            // output if time is less than 1 second
            bpf_trace_printk("%d %d\n", delta / 1000000, count);
        }
        dict.delete(&ts_key);
    }
    
    // update stored timestamp
    ts = bpf_ktime_get_ns();
    dict.update(&ts_key, &ts);
    
    return 0;
}
'''

with BPF(text=prog) as b:
    b.attach_kprobe(event='sys_sync', fn_name='do_trace')
    print('Tracing for quick sync\'...')
    
    start = 0
    with timeout(seconds=20):
        while True:
            task, pid, cpu, flags, ts, msg = b.trace_fields()
            if start == 0:
                start = ts
            ts = ts - start
            ms, count = msg.split()
            print('At time %.2f s: multiple syncs detected, last %s ms ago, total calls: %s' % (ts, ms, count))

In [ ]:
REQ_WRITE = 1

prog = r'''
#include <uapi/linux/ptrace.h>
#include <linux/blkdev.h>

BPF_HASH(start, struct request *);

void trace_start(struct pt_regs *ctx, struct request *req) {
    // stash start timestamp by request ptr
    u64 ts = bpf_ktime_get_ns();
    start.update(&req, &ts);
}

void trace_completion(struct pt_regs *ctx, struct request *req) {
    u64 *tsp, delta;
    
    tsp = start.lookup(&req);
    if (tsp != 0) {
        delta = bpf_time_get_ns() - *tsp;
        bpf_trace_printk("%d %x %d\n", req->__data_len, req->cmd_flags
    }
}
'''